### Item 20: Use None and Docstrings to Specify Dynamic default Arguments

* Sometimes you need to use a non-static type as a `keyword` arguments' default value.

    e.g 
    * Print logging messages that are marked with the time of the logged event.
    * In the case that includes the time, the default arguments are to be reevaluated each time the function is called.

In [ ]:
from datetime import datetime
from datetime import date
import json
from time import sleep

In [ ]:
# dir(datetime)

In [ ]:
now = datetime.now()
now

In [ ]:
def log(message, when=now):
    print(f"{when}: {message}")

In [ ]:
log("Hi there!")

In [ ]:
sleep(0.1)
log("Hi again!")

* Problem 1

    * The timestamps are the same because datetime.now is only executed a single time: when the function is defined.
    * Default argument values are evaluated only once per module load, which usually happens when a program starts up.
    * The convention for achieving the desired result in Python is to provide a default value of `None` and to ducument the actual behavior in the docsting.
        * See `Item 49`: Write Docstrings for Every Function, Class, and Module.
        * When the code sees an argument value of `None`, you allocate the `default value` accordingly.

In [ ]:
def log(message, when=None):
    """Log a message with a timestamp.
    
    Args:
        message: Message to print.
        when: detetime of when the message occurred.
            Defaults to the present time.
    """
    when = datetime.now() if when is None else when
    print(f"{when}: {message}")

* Now the timestamp is different

In [ ]:
log("Hi there!")
sleep(0.1)
log("Hi again!")

* Using `None` for default argument values is especially important when the arguments are mutable.

    * Example: an empty dictionary to be returned by default

In [ ]:
def decode(data, default={}):
    try: 
        return json.loads(data)
    except ValueError:
        return default

* Problem 2
    * Same problem as the example above.
    * Default argument values are only evaluated once (at module load time).
    * The dictionary specified for default will be shared by all calls to decode.
    * This can cause extremely surprising behavior.

In [ ]:
foo = decode('bad data')

In [ ]:
foo['stuff'] = 5

In [ ]:
bar = decode('also bad')

In [ ]:
bar['meep'] = 1

In [ ]:
print('Foo:', foo)
print('Bar:', bar)

* Expect two different dictionaries, each with a single key and value.
* But modifying one seems to also modify the other.
* The `foo` and `bar` are both equal to the default parameter.
    * They are the same dictionary object.

In [ ]:
assert foo is bar

#### The fix 
[important!]

* To set the keyword argument `default` value to `None`.
* And then document the behavior in the function's docstring.

In [ ]:
def decode(data, default=None):
    """Load JSON data from a string.
    
    Args:
        data: JSON data to decode.
        default: Value to return if docoding fails.
            Defaults to an empty dictionary.
    """
    if default is None:
        default = {}
    try:
        return json.loads(data)
    except ValueError:
        return default

* This produces the expected result

In [ ]:
foo = decode('bad data')

In [ ]:
foo['stuff'] = 5

In [ ]:
bar = decode('also bad')

In [ ]:
bar['meep'] = 1

In [ ]:
print("Foo:", foo)
print("Bar:", bar)

### Things to Remember

* `Default` arguments are only evaluated once: during function definition at module load time.
    * This can cause odd behaviors for dynamic values (like {} or [])
* Use `None` as the default value for `keyword` arguments that have a dynamic value.
* document the actual `default` behavior in the function's docstring.